### 1. Import thư viện

In [2]:
import json
from collections import defaultdict

### 2. Khai báo tên file

In [3]:
champion_win_rate_file_name = r'D:\Learning\ML\BTL\preprocessing\clean_data\champion_win_rate.json'
detail_matches_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\detail_match_full.json'

### 3. Tính tỷ lệ thắng của mỗi tướng

In [4]:
with open(detail_matches_file_name, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [24]:
stats = defaultdict(lambda: {
    'wins': 0, 
    'losses': 0, 
    'picks': 0, 
    'bans': 0
})

In [25]:
# Duyệt qua từng trận đấu và tính toán các chỉ số
for match in data:
    team_100_picks = match["team_100_picks"]
    team_200_picks = match["team_200_picks"]
    team_100_bans = match["team_100_bans"]
    team_200_bans = match["team_200_bans"]
    result = match["result"]
    
    if result == 1:  # Team 100 thắng
        for champion_id in team_100_picks:
            stats[champion_id]['wins'] += 1
        for champion_id in team_200_picks:
            stats[champion_id]['losses'] += 1
    elif result == -1:  # Team 200 thắng
        for champion_id in team_100_picks:
            stats[champion_id]['losses'] += 1
        for champion_id in team_200_picks:
            stats[champion_id]['wins'] += 1

    # Cập nhật số trận được pick và bị ban cho mỗi ID
    for pick in team_100_picks + team_200_picks:
        stats[pick]['picks'] += 1
    for ban in team_100_bans + team_200_bans:
        if ban != -1:  # Giả sử ban có giá trị -1 là không có
            stats[ban]['bans'] += 1

# Tính toán tỷ lệ thắng
for champion_id, stat in stats.items():
    total_games = stat['wins'] + stat['losses']
    total_picks = stat['picks']
    total_bans = stat['bans']

    # Tính tỷ lệ thắng
    if total_games > 0:
        stat['win_rate'] = round(stat['wins'] / total_games * 100, 2)
    else:
        stat['win_rate'] = 0.00
    
    # Tính tỷ lệ chọn
    stat['pick_rate'] = round(total_picks / len(data) * 100, 2)

    # Tính tỷ lệ bị cấm
    stat['ban_rate'] = round(total_bans / len(data) * 100, 2)
    
result_dict = {champion_id: stat for champion_id, stat in sorted(stats.items())}


In [27]:
with open(champion_win_rate_file_name, 'w', encoding='utf-8') as file:
    json.dump(result_dict, file, ensure_ascii=False, indent=4)

In [28]:
len(result_dict)

169